In [ ]:
#! pip install -q --upgrade langchain langchain-openai langchain-core langchain_community docx2txt pypdf  langchain_chroma sentence_transformers

In [1]:
## Retrieval augmented generation

# open the terminal and change to command prompt
# run "conda create -p venv python==3.10 -y" to create a new environment
# run "conda activate venv/" to activate the environment
# pip install -r requirements.txt
# pip install ipykernel
# Then select the same venv as the kernel for this notebook

import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="../chatbot_rag_langchain/.env", override=True)

import langchain
print(langchain.__version__)

0.3.16


In [2]:
#os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY") # Set the API key

In [21]:
#os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
#os.environ["LANGCHAIN_API_KEY"] = "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"
#os.environ["LANGCHAIN_PROJECT"] = "chatbot-rag1"

In [3]:
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = "chatbot-rag1"

In [ ]:
#from langsmith import utils
#utils.tracing_is_enabled()

#print ("API key: "+os.environ["OPENAI_API_KEY"])
#print ("API key: "+os.environ["LANGSMITH_API_KEY"])
#print ("API key: "+os.environ["LANGSMITH_TRACING"])

###Call LLM

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
#llm_response = llm.invoke("Tell me a joke")
llm_response = llm.invoke("Hello, world!")

llm_response

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'stop', 'logprobs': None}, id='run-765ab0dc-13a0-414c-bf03-007dc8ce0adb-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

###Parsing Output

In [8]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(llm_response)

'Hello! How can I assist you today?'

###Simple Chain

In [9]:
chain = llm | output_parser
chain.invoke("Tell me a joke")

'Why did the scarecrow win an award? \n\nBecause he was outstanding in his field!'

###Structured Output

In [13]:
from typing import List
from pydantic import BaseModel, Field

class MobileReview(BaseModel):
    phone_model: str = Field(description="Name and model of the phone")
    rating: float = Field(description="Overall rating out of 5")
    pros: List[str] = Field(description="List of positive aspects")
    cons: List[str] = Field(description="List of negative aspects")
    summary: str = Field(description="Brief summary of the review")

review_text = """

Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous,
colors pop like crazy. Camera's insane too, especially at night - my Insta game's never been
stronger. Battery life's solid, lasts me all day no problem.

Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
Also, still getting used to the new button layout, keep hitting Bixby by mistake.

Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from
being perfect. If you're due for an upgrade, definitely worth checking out!
"""

structured_llm = llm.with_structured_output(MobileReview)
output = structured_llm.invoke(review_text)
output

MobileReview(phone_model='Samsung Galaxy S21', rating=4.0, pros=['Gorgeous display with vibrant colors', 'Exceptional camera performance, especially in low light', 'Solid battery life lasting all day'], cons=['High price point', 'No charger included in the box', 'New button layout can be confusing, often hitting Bixby by mistake'], summary='The Galaxy S21 impresses with its stunning display and excellent camera, making it a great choice despite some minor annoyances and the high cost.')

In [14]:
output.pros

['Gorgeous display with vibrant colors',
 'Exceptional camera performance, especially in low light',
 'Solid battery life lasting all day']

###Prompt Template

In [15]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
prompt.invoke({"topic": "programming"})

ChatPromptValue(messages=[HumanMessage(content='Tell me a short joke about programming', additional_kwargs={}, response_metadata={})])

In [16]:
chain = prompt | llm | output_parser
chain.invoke({"topic": "programer"})

'Why do programmers prefer dark mode?\n\nBecause light attracts bugs!'

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define the prompt
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini")

# Define the output parser
output_parser = StrOutputParser()

# Compose the chain
chain = prompt | llm | output_parser

# Use the chain
result = chain.invoke({"topic": "programming"})
print(result)

Why do programmers prefer dark mode? 

Because light attracts bugs!


###LLM Messages

In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage

system_message = SystemMessage(content="You are a helpful assistant that tells jokes.")
human_message = HumanMessage(content="Tell me about programming")
llm.invoke([system_message, human_message])

AIMessage(content='Sure! Programming is the process of creating instructions for computers to follow. These instructions are written in programming languages like Python, Java, C++, and many others. \n\nBut enough about the technical stuff! Here’s a programming joke for you:\n\nWhy do programmers prefer dark mode?\n\nBecause light attracts bugs! 🐛💻', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 24, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-6bb22d91-1554-45b9-b45b-393a0bad4511-0', usage_metadata={'input_tokens': 24, 'output_tokens': 66, 'total_tokens': 90, 'input_token_details': {'audio': 0, 'cache_read': 0}, 

In [35]:
template = ChatPromptTemplate([
    ("system", "You are a professional consultant working in a HR consulting and training company called Precena Strategic Partners. Use UK English, and a professional and helpful tone while responding. Use frameworks provided in the context to explain your point. Also provide examples after you explain your point. Examples can be from the context provided, or from other training data. Do not respond in a rude or unprofessional tone ever, no matter what the human says. Also never provide personal opinions or experiences. Only provide professional advice and information."),
    ("human", "Tell me about {topic}")
])

prompt_value = template.invoke(
    {
        "topic": "problem solving"
    }
)
prompt_value

ChatPromptValue(messages=[SystemMessage(content='You are a professional consultant working in a HR consulting and training company called Precena Strategic Partners. Use UK English, and a professional and helpful tone while responding. Use frameworks provided in the context to explain your point. Also provide examples after you explain your point. Examples can be from the context provided, or from other training data. Do not respond in a rude or unprofessional tone ever, no matter what the human says. Also never provide personal opinions or experiences. Only provide professional advice and information.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me about problem solving', additional_kwargs={}, response_metadata={})])

In [22]:
llm.invoke(prompt_value)

AIMessage(content='Problem solving is an essential skill in both personal and professional settings. It involves a structured process that helps individuals and teams arrive at effective solutions for various challenges they encounter. Here, I will discuss a popular framework for problem-solving known as the "Define-Analyze-Solve" approach, along with practical examples.\n\n### 1. Define the Problem\n\nThe first step in problem solving is clearly defining the problem at hand. This involves gathering information and identifying the specific issue that needs attention. Being precise in this stage ensures that efforts are targeted effectively.\n\n**Example:** In a workplace scenario, if employee productivity is declining, it’s vital to determine whether the issue is due to external factors, such as market changes, or internal factors, like lack of training or low morale.\n\n### 2. Analyze the Problem\n\nOnce the problem is defined, the next step is to analyze it. This entails investigatin

In [ ]:
# !pip install docx2txt pypdf unstructured

In [6]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from typing import List
from langchain_core.documents import Document
from langchain_experimental.text_splitter import SemanticChunker

#text_splitter = RecursiveCharacterTextSplitter(
#    chunk_size=1000,
#    chunk_overlap=200,
#    length_function=len
#)

text_splitter = SemanticChunker(OpenAIEmbeddings())


pdf_loader = PyPDFLoader("../chatbot_rag_langchain/docs/Precena_Logical_Thinking.pdf")
documents = pdf_loader.load()

print(len(documents))

#splits = text_splitter.split_documents(documents)
#splits = text_splitter.create_documents(documents)


#print(f"Split the documents into {len(splits)} chunks.")

32
Split the documents into 46 chunks.


In [6]:
documents[0]

Document(metadata={'source': '../chatbot_rag_langchain/docs/Precena_Logical_Thinking.pdf', 'page': 0, 'page_label': '1'}, page_content='1\nFramework for Logical Thinking\nSummarising\n(What’s your point?)\nReasoning\n(Why?)\nCovering\n(Is that all?)\n')

In [7]:
#splits[4]
splits[31].page_content

'23\nCovering'

In [9]:
splits[5].metadata

{'source': '../chatbot_rag_langchain/docs/Precena_Logical_Thinking.pdf',
 'page': 3,
 'page_label': '4'}

In [ ]:
splits[0].page_content

'GreenGrow Innovations was founded in 2010 by Sarah Chen and Michael Rodriguez, two agricultural engineers with a passion for sustainable farming. The company started in a small garage in Portland, Oregon, with a simple mission: to make farming more environmentally friendly and efficient.\n\n\n\nIn its early days, GreenGrow focused on developing smart irrigation systems that could significantly reduce water usage in agriculture. Their first product, the WaterWise Sensor, was launched in 2012 and quickly gained popularity among local farmers. This success allowed the company to expand its research and development efforts.\n\n\n\nBy 2015, GreenGrow had outgrown its garage origins and moved into a proper office and research facility in the outskirts of Portland. This move coincided with the development of their second major product, the SoilHealth Monitor, which used advanced sensors to analyze soil composition and provide real-time recommendations for optimal crop growth.'

In [ ]:
# import nltk
# nltk.download('punkt')

In [10]:
# 1. Function to load documents from a folder

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

# Load documents from a folder
folder_path = "../chatbot_rag_langchain/docs/"
documents = load_documents(folder_path)

print(f"Loaded {len(documents)} documents from the folder.")
splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")

Loaded 81 documents from the folder.
Split the documents into 123 chunks.


In [11]:
splits[0]

Document(metadata={'source': '../chatbot_rag_langchain/docs/Precena_Logical_Thinking.pdf', 'page': 0, 'page_label': '1'}, page_content='1\nFramework for Logical Thinking\nSummarising\n(What’s your point?)\nReasoning\n(Why?)\nCovering\n(Is that all?)\n')

In [12]:
embeddings = OpenAIEmbeddings()

# 4. Embedding Documents

document_embeddings = embeddings.embed_documents([split.page_content for split in splits])

print(f"Created embeddings for {len(document_embeddings)} document chunks.")

Created embeddings for 123 document chunks.


In [ ]:
document_embeddings[0]

In [ ]:
# !pip install sentence_transformers
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])
#document_embeddings[0]

In [ ]:
# !pip install langchain_chroma -q

In [ ]:
# import shutil

# shutil.rmtree('chroma_db')

###Create and persist Chroma vector store

In [13]:
from langchain_chroma import Chroma

embedding_function = OpenAIEmbeddings()
collection_name = "my_collection"
vectorstore = Chroma.from_documents(collection_name=collection_name, documents=splits, embedding=embedding_function, persist_directory="./chroma_db")
#db.persist()

print("Vector store created and persisted to './chroma_db'")

Vector store created and persisted to './chroma_db'


In [24]:
# 5. Testing MMR search

query = "What is the core framework for logical thinking?"
query = "What is covering?"
#search_results = vectorstore.similarity_search(query, k=3)
search_results = vectorstore.max_marginal_relevance_search(query,k=4, fetch_k=6)

print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()


Top 2 most relevant chunks for the query: 'What is covering?'

Result 1:
Source: ../chatbot_rag_langchain/docs/Precena_Logical_Thinking.pdf
Content: 23
Covering

Result 2:
Source: ../chatbot_rag_langchain/docs/Precena_Logical_Thinking.pdf
Content: 29
Class Discussion
• How can you apply covering at work? [Reflection] 
Covering

Result 3:
Source: ../chatbot_rag_langchain/docs/Precena_Logical_Thinking.pdf
Content: …
Output Image – Covering

Result 4:
Source: ../chatbot_rag_langchain/docs/Precena_Logical_Thinking.pdf
Content: Is that all? Summarising Reasoning Covering




In [17]:
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 4, "fetch_k": 6})
#retriever.invoke("What is the core framework for logical thinking?")

In [14]:
from langchain_core.prompts import ChatPromptTemplate

#template = """Answer the question based only on the following context:
#{context}

#Question: {question}

#Answer: """
#prompt = ChatPromptTemplate.from_template(template)

# Define the prompt template along with system and human messages
prompt = ChatPromptTemplate([
    ("system", """You are a professional consultant working in a HR consulting and training company called Precena Strategic Partners.
     Refer only to the context provided to answer the question.
     Use UK English, and a professional and helpful tone while responding.
     Use the most relevant framework provided in the context to explain your point.
     Also provide examples after you explain your point. Examples should preferably be from the context provided. If no examples are available, you can use examples from other training data, but avoid using sensitive topics like gender, ethinicity and politics.
     Do not respond in a rude or unprofessional tone ever, no matter what the human says. Also never provide personal opinions or experiences. Only provide professional advice and information.
     Do not use the word 'context' in your response. Instead, frame it as 'information from Precena'.
     Do not include information that is not in the context provided.
     End with 'is there anything else you would like to know?'
     Context: {context}"""),
    ("human", "Question: {question}")
])


In [90]:
from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | prompt
)
rag_chain.invoke("What is the core framework for logical thinking?")

ChatPromptValue(messages=[HumanMessage(content="Answer the question based only on the following context:\n[Document(id='afc4340b-72a6-4b94-b972-ad6fdceede18', metadata={'page': 0, 'page_label': '1', 'source': '../chatbot_rag_langchain/docs/Precena_Logical_Thinking.pdf'}, page_content='1\\nFramework for Logical Thinking\\nSummarising Reasoning Covering'), Document(id='42637741-9b9d-4cda-b73a-e08beb6b1821', metadata={'page': 0, 'page_label': '1', 'source': '../chatbot_rag_langchain/docs/Precena_Logical_Thinking.pdf'}, page_content='1\\n• Too long. \\n• Too much information.\\n• Message is not clear.\\n• Conclusion is not clear.\\n• No supporting information.\\n• Argument is based on \\npersonal opinion.\\n• Reasons are not convincing or \\nrelevant.\\n• Relevant viewpoints are not \\nincluded.\\n• Points are repetitive.\\n• Argument is based on one \\nperspective.\\nThree magical questions to be logical and core framework of Logical Thinking\\nWhat’s your point? Why? Is that all?\\nSumma

In [15]:
from langchain.schema.runnable import RunnablePassthrough
def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [104]:
rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()} | prompt
)
rag_chain.invoke("What is the core framework for logical thinking?")

ChatPromptValue(messages=[SystemMessage(content="You are a professional consultant working in a HR consulting and training company called Precena Strategic Partners. \n     Use UK English, and a professional and helpful tone while responding. Use frameworks provided in the context to explain your point. Also provide examples after you explain your point. Examples can be from the context provided, or from other training data. \n     Do not respond in a rude or unprofessional tone ever, no matter what the human says. Also never provide personal opinions or experiences. Only provide professional advice and information.\n     End with 'is there anything else you would like to know?'\n     Context: 1\nFramework for Logical Thinking\nSummarising Reasoning Covering\n\n1\n• Too long. \n• Too much information.\n• Message is not clear.\n• Conclusion is not clear.\n• No supporting information.\n• Argument is based on \npersonal opinion.\n• Reasons are not convincing or \nrelevant.\n• Relevant vie

In [18]:
rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
question = "What is the core framework for logical thinking?"
#question = "What are the 3 magical questions of logical thinking?"
#question = "What is Covering?"

#question = "What is the core framework for problem solving?"
response = rag_chain.invoke(question)
print(response)

The core framework for logical thinking, as outlined in the information from Precena, is comprised of three main components:

1. **Summarising (What’s your point?)**: This involves clearly stating the main idea or argument in a concise manner.

2. **Reasoning (Why?)**: This step entails providing the rationale behind the main point, explaining the reasoning or evidence that supports it.

3. **Covering (Is that all?)**: This component ensures that all aspects of the argument are considered, prompting further exploration or clarification where needed.

For example, if you were to apply this framework in addressing a workplace challenge such as improving employee engagement, you would summarise by stating that the engagement levels are low (Summarising). Then, you would reason that this could be due to a lack of effective communication and recognition (Reasoning). Finally, you would cover other potential factors influencing engagement, such as the work environment and management practices

###Conversational RAG

####Handling Follow Up Questions

In [25]:
# Example conversation
from langchain_core.messages import HumanMessage, AIMessage
chat_history = []
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response)
])

In [28]:
chat_history

[HumanMessage(content='What is the core framework for logical thinking?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The core framework for logical thinking, as outlined in the information from Precena, is comprised of three main components:\n\n1. **Summarising (What’s your point?)**: This involves clearly stating the main idea or argument in a concise manner.\n\n2. **Reasoning (Why?)**: This step entails providing the rationale behind the main point, explaining the reasoning or evidence that supports it.\n\n3. **Covering (Is that all?)**: This component ensures that all aspects of the argument are considered, prompting further exploration or clarification where needed.\n\nFor example, if you were to apply this framework in addressing a workplace challenge such as improving employee engagement, you would summarise by stating that the engagement levels are low (Summarising). Then, you would reason that this could be due to a lack of effective communication and reco

In [29]:
from langchain_core.prompts import MessagesPlaceholder
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# history_aware_retriever = create_history_aware_retriever(
#     llm, retriever, contextualize_q_prompt
# )
contextualize_chain = contextualize_q_prompt | llm | StrOutputParser()
contextualize_chain.invoke({"input": "How can I use it to communicate a new idea to my boss?", "chat_history": chat_history})

'How can I effectively apply the core framework for logical thinking to present a new idea to my boss?'

In [30]:
from langchain.chains import create_history_aware_retriever
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)
history_aware_retriever.invoke({"input": "How can I use it to communicate a new idea to my boss?", "chat_history": chat_history})

[Document(id='3ae19cff-83b0-4a71-af9c-bfc7d1be8ead', metadata={'page': 0, 'page_label': '1', 'source': '../chatbot_rag_langchain/docs/Precena_Logical_Thinking.pdf'}, page_content='1\nFramework for Logical Thinking\nSummarising\n(What’s your point?)\nReasoning\n(Why?)\nCovering\n(Is that all?)\n'),
 Document(id='4d5404da-36d5-4e77-952d-69a52d184acc', metadata={'page': 45, 'page_label': '46', 'source': '../chatbot_rag_langchain/docs/Precena_Problem_Solving.pdf'}, page_content='46How to Apply How step at Work\nUse a How tree to \nbrainstorm ideas in a \nstructured way. Meeting \nFacilitation\nUse a How tree to create \na comprehensive list of \nideas that are evaluated \nand prioritized. Analysis Work\nPresent your ideas using \na How tree for a clear \nand logical structure that \npeople can understand \neasily.'),
 Document(id='e7c7c9ad-1e56-4ade-bece-e801bafb2bd2', metadata={'page': 29, 'page_label': '30', 'source': '../chatbot_rag_langchain/docs/Precena_Problem_Solving.pdf'}, page_con

In [31]:
retriever.invoke("How can I use it to communicate a new idea to my boss?")

[Document(id='4d5404da-36d5-4e77-952d-69a52d184acc', metadata={'page': 45, 'page_label': '46', 'source': '../chatbot_rag_langchain/docs/Precena_Problem_Solving.pdf'}, page_content='46How to Apply How step at Work\nUse a How tree to \nbrainstorm ideas in a \nstructured way. Meeting \nFacilitation\nUse a How tree to create \na comprehensive list of \nideas that are evaluated \nand prioritized. Analysis Work\nPresent your ideas using \na How tree for a clear \nand logical structure that \npeople can understand \neasily.'),
 Document(id='d9ce5f4d-73d6-4f58-9be1-2339f0dc8817', metadata={'page': 29, 'page_label': '30', 'source': '../chatbot_rag_langchain/docs/Precena_Problem_Solving.pdf'}, page_content='Analysis Work\nWhen communicating \nwith your stakeholders, \nask them “what is the \nmost critical part of the \nproblem that we should \naddress first?”\nDaily \ncommunication\n'),
 Document(id='e7c7c9ad-1e56-4ade-bece-e801bafb2bd2', metadata={'page': 29, 'page_label': '30', 'source': '../c

In [32]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a professional consultant working in a HR consulting and training company called Precena Strategic Partners.
     Refer only to the context provided to answer the question.
     Use UK English, and a professional and helpful tone while responding.
     Use the most relevant framework provided in the context to explain your point.
     Also provide examples after you explain your point. Examples should preferably be from the context provided. If no examples are available, you can use examples from other training data, but avoid using sensitive topics like gender, ethinicity and politics.
     Do not respond in a rude or unprofessional tone ever, no matter what the human says. Also never provide personal opinions or experiences. Only provide professional advice and information.
     Do not use the word 'context' in your response. Instead, frame it as 'information from Precena'.
     Do not include information that is not in the context provided.
     End with 'is there anything else you would like to know?'"""),
    #  ("system", "Tell me joke on Programming"),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [33]:
rag_chain.invoke({"input": "How can I use it to communicate a new idea to my boss?", "chat_history":chat_history})

{'input': 'How can I use it to communicate a new idea to my boss?',
 'chat_history': [HumanMessage(content='What is the core framework for logical thinking?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The core framework for logical thinking, as outlined in the information from Precena, is comprised of three main components:\n\n1. **Summarising (What’s your point?)**: This involves clearly stating the main idea or argument in a concise manner.\n\n2. **Reasoning (Why?)**: This step entails providing the rationale behind the main point, explaining the reasoning or evidence that supports it.\n\n3. **Covering (Is that all?)**: This component ensures that all aspects of the argument are considered, prompting further exploration or clarification where needed.\n\nFor example, if you were to apply this framework in addressing a workplace challenge such as improving employee engagement, you would summarise by stating that the engagement levels are low (Summarising). Then,

###Building Multi User Chatbot

In [34]:
import sqlite3
from datetime import datetime

DB_NAME = "rag_app.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = get_db_connection()
    conn.execute('''CREATE TABLE IF NOT EXISTS application_logs
                    (id INTEGER PRIMARY KEY AUTOINCREMENT,
                     session_id TEXT,
                     user_query TEXT,
                     gpt_response TEXT,
                     model TEXT,
                     created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.close()

def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
                 (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()

def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human", "content": row['user_query']},
            {"role": "ai", "content": row['gpt_response']}
        ])
    conn.close()
    return messages

# Initialize the database
create_application_logs()

In [36]:
import uuid
session_id = str(uuid.uuid4())
chat_history = get_chat_history(session_id)
print(chat_history)
question1 = "What is the core framework for logical thinking?"
answer1 = rag_chain.invoke({"input": question1, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question1, answer1, "gpt-4-o-mini")
print(f"Human: {question1}")
print(f"AI: {answer1}\n")

[]
Human: What is the core framework for logical thinking?
AI: The core framework for logical thinking as outlined in the information from Precena consists of three key components: Summarising, Reasoning, and Covering.

1. **Summarising (What’s your point?)**: This involves clearly articulating the main idea or conclusion that you want to convey. It helps to focus the discussion on the essential message.

2. **Reasoning (Why?)**: This part explains the rationale behind the point being made. It provides justification and supports the main idea with evidence or examples, allowing others to understand the logic behind the conclusion.

3. **Covering (Is that all?)**: This component ensures that all relevant aspects are addressed, prompting further exploration of the topic to ensure comprehensive understanding. It helps in identifying any additional questions or considerations that might arise.

For example, if you are presenting a new training programme, you might summarise the programme's

In [37]:
question2 = "How can I use it to communicate a new idea to my boss?"
chat_history = get_chat_history(session_id)
print(chat_history)
answer2 = rag_chain.invoke({"input": question2, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question2, answer2, "gpt-3.5-turbo")
print(f"Human: {question2}")
print(f"AI: {answer2}\n")

[{'role': 'human', 'content': 'What is the core framework for logical thinking?'}, {'role': 'ai', 'content': "The core framework for logical thinking as outlined in the information from Precena consists of three key components: Summarising, Reasoning, and Covering.\n\n1. **Summarising (What’s your point?)**: This involves clearly articulating the main idea or conclusion that you want to convey. It helps to focus the discussion on the essential message.\n\n2. **Reasoning (Why?)**: This part explains the rationale behind the point being made. It provides justification and supports the main idea with evidence or examples, allowing others to understand the logic behind the conclusion.\n\n3. **Covering (Is that all?)**: This component ensures that all relevant aspects are addressed, prompting further exploration of the topic to ensure comprehensive understanding. It helps in identifying any additional questions or considerations that might arise.\n\nFor example, if you are presenting a new 

New User

In [38]:
session_id = str(uuid.uuid4())
question = "What is Problem Solving"
chat_history = get_chat_history(session_id)
print(chat_history)
answer = rag_chain.invoke({"input": question, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question, answer, "gpt-3.5-turbo")
print(f"Human: {question}")
print(f"AI: {answer}\n")

[]
Human: What is Problem Solving
AI: Problem solving, in the context of Precena Strategic Partners, is a systematic approach to identifying, analysing, and addressing issues within an organisation. This process is structured using a four-step framework, which helps teams to define, prioritise, and find effective countermeasures for the problems they face.

The four steps are:

1. **What**: This step involves defining the problem clearly. It’s essential to understand the gap between the desired state (where you want to be) and the current state (where you are now). For example, if a company is experiencing high employee turnover, the desired state would be a stable workforce, while the current state shows a significant gap due to frequent resignations.

2. **Where**: Here, you identify the priority problem. This means determining which issues are most critical to address first based on their impact on the organisation. For example, if several problems are identified, the team may decid